In [1]:
import os
import string
import random
import numpy as np
from sklearn.model_selection import KFold
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from modnet.models import EnsembleMODNetModel
from modnet.preprocessing import MODData
from modnet.hyper_opt import FitGenetic
from monty.serialization import dumpfn, loadfn
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

2023-11-30 15:31:50.422547: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-30 15:31:50.424176: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 15:31:50.447292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 15:31:50.447322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 15:31:50.448030: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import os
import string
import random
import numpy as np
from sklearn.model_selection import KFold
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from modnet.models import EnsembleMODNetModel
from modnet.preprocessing import MODData
from modnet.hyper_opt import FitGenetic
from monty.serialization import dumpfn, loadfn
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Loading MODData from "experiments"

In [3]:
data: MODData = MODData.load("../../../../re2fractive_onedrive/data/mod.data_feature_selected.pkl")

2023-11-30 15:31:55,042 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f9cb3a7ee80> object, created with modnet version 0.2.1


In [4]:
data.df_featurized

,AtomicOrbitals|HOMO_character,AtomicOrbitals|HOMO_element,AtomicOrbitals|HOMO_energy,AtomicOrbitals|LUMO_character,AtomicOrbitals|LUMO_element,AtomicOrbitals|LUMO_energy,AtomicOrbitals|gap_AO,AtomicPackingEfficiency|mean simul. packing efficiency,AtomicPackingEfficiency|mean abs simul. packing efficiency,AtomicPackingEfficiency|dist from 1 clusters |APE| < 0.010,...,VoronoiFingerprint|mean Voro_area_maximum,VoronoiFingerprint|std_dev Voro_area_maximum,VoronoiFingerprint|mean Voro_dist_mean,VoronoiFingerprint|std_dev Voro_dist_mean,VoronoiFingerprint|mean Voro_dist_std_dev,VoronoiFingerprint|std_dev Voro_dist_std_dev,VoronoiFingerprint|mean Voro_dist_minimum,VoronoiFingerprint|std_dev Voro_dist_minimum,VoronoiFingerprint|mean Voro_dist_maximum,VoronoiFingerprint|std_dev Voro_dist_maximum
id,,,,,,,,,,,,,,,,,,,,,
mp-755116,3.0,78,-0.273634,3.0,78,-0.273634,0.000000,-0.031815,0.031815,0.064282,...,4.400785,0.132124,2.736224,0.208690,0.467059,0.210464,2.064988,0.078563,3.377776,0.458776
mp-755478,2.0,8,-0.338381,2.0,8,-0.338381,0.000000,0.005285,0.013314,0.020412,...,6.704337,0.361393,3.150719,0.185900,0.630736,0.106049,2.219102,0.068728,4.084165,0.184609
mp-17998,3.0,45,-0.239422,3.0,45,-0.239422,0.000000,0.019235,0.032468,0.053452,...,4.414231,0.340455,2.716448,0.163359,0.463026,0.088636,2.120453,0.112051,3.501111,0.171479
mp-754065,2.0,8,-0.338381,2.0,8,-0.338381,0.000000,-0.028964,0.030704,0.033259,...,5.273731,0.608247,2.837959,0.437157,0.465939,0.250315,2.069003,0.132846,3.796428,0.976473
mp-553961,2.0,8,-0.338381,1.0,21,-0.156478,0.181903,-0.019762,0.033897,0.017750,...,5.049319,0.215777,2.753045,0.260710,0.518376,0.048432,1.985772,0.073579,3.874981,0.420384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-698480,2.0,8,-0.338381,2.0,8,-0.338381,0.000000,0.000000,0.000000,0.146979,...,6.755468,1.877188,3.122917,0.205950,0.800698,0.131879,1.619887,0.417271,4.356481,0.277054
mp-23740,2.0,8,-0.338381,2.0,8,-0.338381,0.000000,0.000000,0.000000,0.134658,...,5.763350,0.537519,2.868246,0.163738,0.677551,0.116411,1.523910,0.349452,3.742030,0.160967
mp-8333,2.0,9,-0.415606,1.0,50,-0.369349,0.046257,0.000000,0.000000,0.772358,...,5.565581,0.726489,3.043532,0.446536,0.534483,0.195537,2.137459,0.299796,4.097413,0.772753


# New training MODData

## New featurization

In [5]:
md_naccarato = MODData(data.df_structure.values.flatten())

2023-11-30 15:31:55,194 - modnet - INFO - Loaded Matminer2023Featurizer featurizer.


In [6]:
# md_naccarato.featurizer.featurizer_mode = "multi"
# md_naccarato.featurize(n_jobs=6)
# md_naccarato.save("mod.data_refeat_v0")

2023-11-30 15:32:00,272 - modnet - INFO - Computing features, this can take time...
2023-11-30 15:32:00,275 - modnet - INFO - Applying composition featurizers...
2023-11-30 15:32:00,410 - modnet - INFO - Applying featurizers (AtomicOrbitals(), AtomicPackingEfficiency(), BandCenter(), ElementFraction(), ElementProperty(data_source=<matminer.utils.data.MagpieData object at 0x7f9c3a39ed90>,
                features=['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber'],
                stats=['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode']), IonProperty(),

MultipleFeaturizer:   1%|          | 30/4001 [00:00<00:13, 298.43it/s]/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/composition/alloy.py:198: RuntimeWarning: invalid value encountered in power
  alp_a = np.multiply(1.5, np.power(v_a, 2 / 3)) / reduce(lambda x, y: 1 / x + 1 / y, np.power(n_ws, 1 / 3))
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/composition/alloy.py:198: RuntimeWarning: invalid value encountered in power
  alp_a = np.multiply(1.5, np.power(v_a, 2 / 3)) / reduce(lambda x, y: 1 / x + 1 / y, np.power(n_ws, 1 / 3))
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/composition/alloy.py:198: RuntimeWarning: invalid value encountered in power
  alp_a = np.multiply(1.5, np.power(v_a, 2 / 3)) / reduce(lambda x, y: 1 / x + 1 / y, np.power(n_ws, 1 / 3))
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/composit

2023-11-30 15:44:47,820 - modnet - INFO - Applying oxidation state featurizers...


CompositionToOxidComposition: 100%|██████████| 4001/4001 [02:27<00:00, 27.19it/s]


2023-11-30 15:48:28,742 - modnet - INFO - Applying featurizers (ElectronegativityDiff(stats=['minimum', 'maximum', 'range', 'mean', 'std_dev']), OxidationStates(stats=['minimum', 'maximum', 'range', 'std_dev'])) to column 'composition_oxid'.


...

2023-11-30 15:48:36,869 - modnet - INFO - Applying structure featurizers...
2023-11-30 15:48:36,875 - modnet - INFO - Applying featurizers (DensityFeatures(), GlobalSymmetryFeatures(), RadialDistributionFunction(), CoulombMatrix(), SineCoulombMatrix(), EwaldEnergy(), BondFractions(), StructuralHeterogeneity(), MaximumPackingEfficiency(), ChemicalOrdering(), XRDPowderPattern(pattern_length=128)) to column 'structure'.


MultipleFeaturizer:   0%|          | 10/4001 [00:00<00:41, 97.09it/s]/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/pymatgen/analysis/local_env.py:4155: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/pymatgen/analysis/local_env.py:3949: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/structure/matrix.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  zeros[: len(eigs)] = eigs
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/pymatgen/analysis/local_env.py:4155: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation st

2023-11-30 16:50:06,172 - modnet - INFO - Applying site featurizers...


SiteStatsFingerprint:   0%|          | 1/4001 [00:00<12:48,  5.21it/s]/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/site/bonding.py:245: RuntimeWarning: invalid value encountered in arccos
  if np.isnan(np.arccos(dot)):
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/site/bonding.py:245: RuntimeWarning: invalid value encountered in arccos
  if np.isnan(np.arccos(dot)):
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/site/bonding.py:245: RuntimeWarning: invalid value encountered in arccos
  if np.isnan(np.arccos(dot)):
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/site/bonding.py:245: RuntimeWarning: invalid value encountered in arccos
  if np.isnan(np.arccos(dot)):
/home/vtrinquet/anaconda3/envs/modnenv_v2/lib/python3.9/site-packages/matminer/featurizers/site/bonding.py:245: RuntimeWarning: invalid value encounte

2023-11-30 21:53:36,221 - modnet - INFO - Data has successfully been featurized!
2023-11-30 21:53:37,692 - modnet - INFO - Data successfully saved as mod.data_refeat_v0!


In [7]:
path_df_mp_2023_feat = (
    Path("./../../../../") / "re2fractive_onedrive" / "data" / "mp_2023_df_featurized_multi.pkl"
)
df_mp_2023_feat = pd.read_pickle(path_df_mp_2023_feat)

In [8]:
md_naccarato.df_featurized = md_naccarato.df_featurized.filter(items=df_mp_2023_feat.columns.values.flatten(), axis=1)

In [21]:
md_naccarato.df_targets = data.df_targets
md_naccarato_v2 = MODData(
    materials=data.structures,
    targets=data.targets,
    target_names=["refractive_index"],
    structure_ids=data.structure_ids,
)

2023-11-30 22:05:26,854 - modnet - INFO - Loaded Matminer2023Featurizer featurizer.


In [24]:
featurized_df = md_naccarato.df_featurized
featurized_df.index = data.structure_ids
md_naccarato_v2.df_featurized = md_naccarato.df_featurized

In [27]:
# md_naccarato_v2.save("mod.data_refeat_v0_v2")

2023-11-30 22:07:44,350 - modnet - INFO - Data successfully saved as mod.data_refeat_v0_v2!


In [28]:
md_naccarato_v2.feature_selection(n=-1,
                       use_precomputed_cross_nmi=False,
                       n_jobs=6)
# md_naccarato_v2.save("mod.data_refeatselec_v0_v2")

2023-11-30 22:08:12,762 - modnet - INFO - Multiprocessing on 6 workers.
2023-11-30 22:08:12,764 - modnet - INFO - Computing "self" MI (i.e. information entropy) of features


100%|██████████| 3019/3019 [00:08<00:00, 356.83it/s]

2023-11-30 22:08:21,336 - modnet - INFO - Computing cross NMI between all features...



100%|██████████| 531996/531996 [36:28<00:00, 243.04it/s]


2023-11-30 22:44:52,889 - modnet - INFO - Starting target 1/1: refractive_index ...
2023-11-30 22:44:52,889 - modnet - INFO - Computing mutual information between features and target...
2023-11-30 22:45:35,955 - modnet - INFO - Computing optimal features...
2023-11-30 22:45:46,638 - modnet - INFO - Selected 50/974 features...
2023-11-30 22:45:57,557 - modnet - INFO - Selected 100/974 features...
2023-11-30 22:46:08,507 - modnet - INFO - Selected 150/974 features...
2023-11-30 22:46:19,408 - modnet - INFO - Selected 200/974 features...
2023-11-30 22:46:30,132 - modnet - INFO - Selected 250/974 features...
2023-11-30 22:46:40,547 - modnet - INFO - Selected 300/974 features...
2023-11-30 22:46:50,905 - modnet - INFO - Selected 350/974 features...
2023-11-30 22:47:00,807 - modnet - INFO - Selected 400/974 features...
2023-11-30 22:47:10,274 - modnet - INFO - Selected 450/974 features...
2023-11-30 22:47:19,219 - modnet - INFO - Selected 500/974 features...
2023-11-30 22:47:27,510 - modnet 

In [7]:
md = MODData.load("mod.data_featselec_opti_v0")

KeyboardInterrupt: 